<!--
tags: [tutorial]
description: Conversation patterns
-->

# Conversation Patterns

In the [last chapter](./code-executors) we used two-agent conversation, which
can be started by the `initiate_chat` method.
Two-agent chat is a useful conversation pattern but AutoGen offers more.
In this chapter, we will first dig a little bit more into the two-agent 
chat pattern and chat result, 
then we will show you several conversation patterns that involve 
more than two agents.


In [1]:
import os
from autogen import ConversableAgent, GroupChat, GroupChatManager

Neither powershell nor pwsh is installed.


## Two-Agent Chat and Chat Result

Two-agent chat is the simplest from of conversation pattern. We have already
seen multiple examples of two-agent chat in the previous chapters
but there are still some details that we haven't covered yet.
The following figure illustrates how two-agent chat works.

```{=mdx}
![Two-agent chat](./assets/two-agent-chat.png)
```

Besides the conversation itself, there is also a summarizer that can be used
to produce a summary of the conversation. The summarizer is optional and by
default the summary is the last message of the conversation.
This can be changed by setting the `summarizer` parameter of the `initiate_chat`.
The example below shows how to use an LLM-based summarizer.

In [7]:
agent_a = ConversableAgent(
    name="Agent A",
    system_message="You are a student willing to learn.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)
agent_b = ConversableAgent(
    name="Agent B",
    system_message="You are a math teacher.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

chat_result = agent_a.initiate_chat(
    agent_b,
    message="What is triangle inequality?",
    max_turns=2,
    summary_method="reflection_with_llm",
)

Agent A (to Agent B):

What is triangle inequality?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent B (to Agent A):

Triangle inequality theorem is a fundamental principle in geometry that states that the sum of the lengths of any two sides of a triangle must always be greater than the length of the third side. In a triangle with sides of lengths a, b, and c, the theorem can be written as:

a + b > c
a + c > b
b + c > a

Each of these represents the condition for one specific side (a, b, or c). All must be true for a triangle to exist.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent A (to Agent B):

Thank you for the explanation. This theorem helps in understanding the basic properties of a triangle. It can also be useful when solving geometric problems or proving other mathematical theorems. Can you give me an example of how we can use the triangl

Let's see what the summary looks like.
The summary is stored in the `chat_result` object of type `ChatResult` 
that was returned by the `initiate_chat` method.

In [9]:
print(chat_result.summary)

The triangle inequality theorem states that in a triangle, the sum of the lengths of any two sides must always be greater than the length of the third side. This principle is significant in geometry and is used in solving problems or proving theorems. For instance, if given three line segments, you can determine if they can form a triangle using this theorem.


In the above example, the summarizer is set to `reflection_with_llm` which
takes a list of messages from the conversation and summarize using a call to
an LLM.
The summarizer first tries to use the recipient's LLM, if it is not available
then it uses the sender's LLM. In this case the recipient is "Agent B" and
the sender is "Agent A".
The input prompt for the LLM is the following default prompt:

In [8]:
print(ConversableAgent.DEFAULT_summary_prompt)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


You can also use a custom prompt by setting the `summary_prompt` argument
of `initiate_chat`.

There are some other useful information in the `chat_result` object, including
the conversation history, human input, and token cost.

In [10]:
# Get the chat history.
print(chat_result.chat_history)

[{'content': 'What is triangle inequality?', 'role': 'assistant'}, {'content': 'Triangle inequality theorem is a fundamental principle in geometry that states that the sum of the lengths of any two sides of a triangle must always be greater than the length of the third side. In a triangle with sides of lengths a, b, and c, the theorem can be written as:\n\na + b > c\na + c > b\nb + c > a\n\nEach of these represents the condition for one specific side (a, b, or c). All must be true for a triangle to exist.', 'role': 'user'}, {'content': 'Thank you for the explanation. This theorem helps in understanding the basic properties of a triangle. It can also be useful when solving geometric problems or proving other mathematical theorems. Can you give me an example of how we can use the triangle inequality theorem?', 'role': 'assistant'}, {'content': 'Absolutely! Here\'s an example:\n\nSuppose you\'re given three line segments with lengths 10, 7, and 3 units. The question is: "Can these three l

In [11]:
# Get the cost of the chat.
print(chat_result.cost)

({'total_cost': 0.04521, 'gpt-4-0613': {'cost': 0.04521, 'prompt_tokens': 709, 'completion_tokens': 399, 'total_tokens': 1108}}, {'total_cost': 0})


## Sequential Two-Agent Chats

The name of this pattern is self-explanatory -- it is a sequence of
two-agent chats, chained together by a mechanism called *carryover*,
which brings the summary of the previous chat to the initial context
of the next chat.

This pattern is useful for complex task that can be broken down into 
interdependent sub-tasks. 
The figure below illustrate how this pattern works.

```{=mdx}
![initiate_chats](./assets/sequential-two-agent-chat.png)
```

In this pattern, the a pair of agents first start a conversation,
then the summary of the conversation is inserted into the initial
message of the next conversation. The previous summary is called a *carryover*.
Carryover accumulates as the conversation goes on, 
so each subsequent conversation is started with all the carryovers 
from previous conversations.

The figure above shows distinct recipient agents for all the chats, however,
the recipient agents in the sequence are allowed to repeat.

To illustrate this pattern, let's consider a simple example of arithmetic
operator agents. One agent 
(called the "Number_Agent") is responsible for coming up with a number,
and other agents are responsible for performing a specific arithmetic
operation on the number, e.g., add 1, multiply by 2, etc..
The Number Agent chats with the first operator agent, then the second
operator agent, and so on.
In the end we will have the result of the arithmetic operations.

In [13]:
# The Number Agent that always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Adder Agent that adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Multiplier Agent that multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Subtracter Agent that subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Divider Agent that divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
14

With the following carryover: 


********************************************************************************
Number_Agent (to Adder_Agent):

14

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_Agent):

15

--------------------------------------------------------------------------------


Adder_Agent (to Number_Agent):

16

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
These are my numbers

With the following carryover: 
16

********************************************************************************
Number_Agent (to Multiplier_Agent):

These are my numbers
Context: 
16

--------------------------------------------------------------------------------
Multiplier_Agent (to Number_Agent):

32

--------------------------------------------------------------------------------
Number_Agent (to Multiplier_Agent):

32

--------------------------------------------------------------------------------
Multiplier_Agent (to Number_Agent):

64

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat

First thing to note is that the `initiate_chats` method takes a list of
dictionaries, each dictionary contains the arguments for the `initiate_chat`
method.

Second, each chat in the sequence has a maximum round of 2, as specified with
the setting `max_turns=2`, 
which means each arithmetic operation is performed twice.
So you can see in the first chat the number 14 becomes 15 and then 16,
in the second chat the number 16 becomes 32 and then 64, and so on.

Third, the carryover accumulates as the chats go on. In the second chat,
the carryover is the summary of the first chat, which is "16".
In the third chat, the carryover is the summary of the first and second
chat, which is the list "16" and "64", and both numbers are operated upon.
In the forth and last chat, the carryover is the summary of all previous
chats, which is the list "16", "64", "14" and "62", and all of these
numbers are operated upon.

The final note is that the `initiate_chats` method returns a list of
`ChatResult` objects, one for each chat in the sequence.

In [15]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  16
Second Chat Summary:  64
Third Chat Summary:  14
62
Fourth Chat Summary:  4
16
3.5
15.5


Besides calling `initiate_chats` from the same sender agent, you can also
call a high-level function `autogen.agentchat.initiate_chats` to start
a sequence of two-agent chats with different sender agents. 
This function allows you to specify the
sender agent for each chat in the sequence.

## Group Chat

So far we have only seen conversation patterns that involve two agents or 
a sequence of two-agent chats. AutoGen provides a more general conversation
pattern called group chat, which involves more than two agents.
The core idea of group chat is that all agents contribute to a single
conversation thread and share the same context.
This is useful for tasks that require collaboration among multiple agents.

The figure below illustrates how group chat works.

```{=mdx}
![group_chat](./assets/group-chat.png)
```

A group chat is orchestrated by a special
agent type `GroupChatManager`. In the first step of the group chat,
the Group Chat Manager selects an agent to speak. Then, the selected agent
speaks and the message is sent back to the Group Chat Manager,
who **broadcasts** the message to all other agents in the group.
This process repeats until the conversation stops.

In [3]:
adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent],
    messages=[],
    max_round=16,
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 40, I want to turn it into 12.",
)

Number_Agent (to chat_manager):

My number is 40, I want to turn it into 12.

--------------------------------------------------------------------------------
Divider_Agent (to chat_manager):

20

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

19

--------------------------------------------------------------------------------
Divider_Agent (to chat_manager):

9.5

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

10.5

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

21

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

20

--------------------------------------------------------------------------------
Divider_Agent (to chat_manager):

10

-------------------------------------------------------------------------------

## Nested Chat

[To be completed...]